In [ ]:
%load_ext autoreload
%autoreload 2
%store -r the_editor
the_editor.to_frame('value')

# Editions per editor with Wikiwho

In [ ]:
from wikiwho_wrapper import WikiWho
import pandas as pd
wikiwho = WikiWho(lng='en')
editions = wikiwho.dv.editions(editor_id = the_editor.userid)
editions.head()

In [ ]:
editor_editions = editions
editor_editions['year_month'] = pd.to_datetime(editor_editions['year_month'])

# Group the data by year month and page (drop the editor information)
editor_editions.drop('editor_id', axis=1).groupby(['year_month','page_id']).sum()

# add columns with the total actions
editor_editions = editor_editions.join(pd.DataFrame(
    editor_editions.loc[:,'adds':'adds_stopword_count'].values +\
    editor_editions.loc[:,'dels':'dels_stopword_count'].values +\
    editor_editions.loc[:,'reins':'reins_stopword_count'].values, 
    index=editor_editions.index, 
    columns=['actions', 
             'actions_surv_48h', 
             'actions_persistent', 
             'actions_stopword_count']
))

# Visualization
from visualization.editions_listener import DFListener
from ipywidgets import interact
listener = DFListener(editor_editions)
actions = (editor_editions.loc[:,'actions':'actions_stopword_count'].columns.append(
    editor_editions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()


interact(listener.editions_per_month, 
         begin=editor_editions.year_month,
         end=editor_editions.year_month.sort_values(ascending=False),
         granularity=['Monthly', 'Yearly'],
         black=actions, red=['actions_surv_48h'] + actions + ['None'],
         green=['None'] + actions, blue=['None'] + actions)


In [ ]:
from IPython.display import HTML
HTML('<a href="6. General metadata of the editor.ipynb" target="_blank">Go to next workbook</a>')